In [ ]:
%matplotlib notebook
%load_ext rpy2.ipython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


nodes = np.arange(10, 101, 10)
solvers = ["cplex", "cbc", "lpSolve"]

def plot_benchmarks(value, num_nodes, ilp_solvers, **kwargs):
    get_benchmark = lambda x, y : pd.read_csv(f"Output/N{x}_I3_M3_S1_P0.2/{y}/benchmark.tsv", sep="\t")
    data = [[get_benchmark(x, y)[value].values for x in num_nodes] for y in ilp_solvers]

    fig, axs = plt.subplots(1, len(solvers), figsize=(12, 5), sharey=True)
    for i, ax in enumerate(axs):
        ax.boxplot(data[i])
        ax.set_xticklabels(num_nodes)
        ax.set_xlabel("Number of nodes")
        ax.set_title(ilp_solvers[i])
        
    if "ylabel" in kwargs:
        axs[0].set_ylabel(kwargs["ylabel"])


In [ ]:
plot_benchmarks("s", nodes, solvers, ylabel="Execution time [s]")

In [ ]:
plot_benchmarks("max_vms", nodes, solvers, ylabel="Memory [B]")

In [ ]:
%%R
print(readRDS("Output/N100_I3_M3_S1_P0.2/cplex/result.Rds")$weightedSIF)
print(readRDS("Output/N100_I3_M3_S1_P0.2/cbc/result.Rds")$weightedSIF)
print(readRDS("Output/N100_I3_M3_S1_P0.2/lpSolve/result.Rds")$weightedSIF)